In [37]:
import requests
import codecs
import csv
from bs4 import BeautifulSoup 
import json
import io
import gzip
from html.parser import HTMLParser

In [50]:
get_requests1 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-09-index?url=http%3A%2F%2Fhttps://www.nfl.com&output=json')
get_requests2 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-04-index?url=http%3A%2F%2Fhttps://www.nfl.com&output=json')
get_requests3 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fhttps://www.nfl.com&output=json')
requestsArray = []
requestsArray.append(get_requests1)
requestsArray.append(get_requests2)
requestsArray.append(get_requests3)


In [51]:
pageList = []
for r in requestsArray:
    print(r.status_code)
    if r.status_code == 200:
        
        pages = r.content.splitlines()
        for i in pages:
            pageList.append(json.loads(i))

print(pages)
#pages = [json.loads(x) for x in get_requests.content.decode('utf8').strip().split('\n')]
#page = pages[2]

200
200
200
[b'{"urlkey": "com,nfl)/", "timestamp": "20190319085609", "digest": "3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ", "offset": "11323229", "length": "555", "url": "http://www.nfl.com/", "filename": "crawl-data/CC-MAIN-2019-13/segments/1552912201922.85/crawldiagnostics/CC-MAIN-20190319073140-20190319095140-00288.warc.gz", "mime": "unk", "status": "301", "mime-detected": "application/octet-stream"}', b'{"urlkey": "com,nfl)/", "timestamp": "20190319085610", "digest": "346AHKUJF3UGDMQYKJ4BTEFZLZFWGTCG", "offset": "835975358", "length": "23779", "url": "https://www.nfl.com/", "languages": "eng", "filename": "crawl-data/CC-MAIN-2019-13/segments/1552912201922.85/warc/CC-MAIN-20190319073140-20190319095140-00111.warc.gz", "mime": "text/html", "charset": "UTF-8", "status": "200", "mime-detected": "text/html"}', b'{"urlkey": "com,nfl)/", "timestamp": "20190319215938", "digest": "EWKG6CDFVJBS6X7YRSVPVUM6LLIIV6HY", "offset": "877394508", "length": "24008", "url": "https://www.nfl.com/", "languages":

In [52]:
page = pageList[0]
print(page)

{'urlkey': 'com,nfl)/', 'timestamp': '20190215230006', 'digest': '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ', 'offset': '10629915', 'length': '556', 'url': 'http://www.nfl.com/', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/crawldiagnostics/CC-MAIN-20190215224408-20190216010408-00048.warc.gz', 'mime': 'unk', 'status': '301', 'mime-detected': 'application/octet-stream'}


In [53]:
offset, length = int(page['offset']), int(page['length'])
offset_end = offset + length - 1
print(offset, length, offset_end)

#prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
prefix = 'https://commoncrawl.s3.amazonaws.com/'

response = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
#response = requests.get("https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/wet/CC-MAIN-20190215224408-20190216010408-00271.warc.wet.gz", headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
print(response)

10629915 556 10630470
<Response [206]>


In [54]:
raw_data = io.BytesIO(response.content)
print(raw_data)
f = gzip.GzipFile(fileobj=raw_data)
print(f)

<gzip _io.BytesIO object at 0x11087c620 0x1108e8780>


In [55]:
data = f.read()

In [56]:
#doc = parse(io.BytesIO(data)).getroot()
doc = BeautifulSoup(io.BytesIO(data), 'lxml')
#print(doc)
all_a = doc.find_all('a')
hrefList = []
if all_a:
    for l in all_a:
        #print(l)
        href = l.attrs.get('href')
        
        if href is not None:
            if href.startswith('http'):
                hrefList.append(href)
                print(href)

In [57]:
with codecs.open('links.csv', 'wb', encoding = 'utf8') as output:
    f = ['url']
    write_csv = csv.DictWriter(output, fieldnames = f)
    write_csv.writeheader()
    
    for h in hrefList:
        write_csv.writerow({'url':h})
        
hrefList = list( dict.fromkeys(hrefList) )

for h in hrefList:
    print(h)
print(hrefList)

[]


In [25]:
#hrefList[1]
all_text = ""
for h in hrefList:
     print("---------------")
     print(h)
     response = requests.get(h)
     if response.status_code == 200:
         doc = BeautifulSoup(response.text, 'html.parser')
         name_box = doc.find_all('p')

         for name in name_box:
            for x in name.find_all('a'):
                x.replaceWithChildren()

         for link in name_box:
            all_text+=link.text
     #print("----------------------------------------------------")
     #print(all_text)

print(all_text)
    
#p = requests.get(hrefList[1])
#print(p.status_code)
#print(p.text)
'''pageListFinal = []
if p.status_code == 200:
    pagesFinal = p.content
    print(pagesFinal)
    for i in pagesFinal:
        pageListFinal.append(json.loads(i))'''
        
#print(pageListFinal[0])

'pageListFinal = []\nif p.status_code == 200:\n    pagesFinal = p.content\n    print(pagesFinal)\n    for i in pagesFinal:\n        pageListFinal.append(json.loads(i))'

In [36]:
print(response.status_code)

206


In [27]:
#print(all_text)
text_file = open("Common_crawl_sbnation_com_text_11_Apr.txt", "w")
text_file.write( all_text)
text_file.close()

In [16]:
 response = requests.get("http://shop.sportingnews.com/COLLEGE")
 doc = BeautifulSoup(response.text, 'html.parser')
 #print(doc)
    
 name_box = doc.find_all('p')
 all_text = ""
 for name in name_box:
    #print(".................")
    #print(name)
    
    for x in name.find_all('a'):
        x.replaceWithChildren()
        #all_text += (x.get_text()) 
        #print(all_text)
    #print("%%%%%%%%%%%")
    #print(name)
    #all_text += name
    #print(all_text)

for link in name_box:
    all_text+=link.text
print(all_text)

The Villanova Wildcats reigned supreme in March Madness, so gear up for the champs with our wide range of NCAA Basketball Apparel including official Villanova National Champs Gear that allow Wildcats fans to ring in their second title in three years! Score more than just College Basketball Gear at Fanatics, as we're always to stock your closet with NCAA Football Apparel and College Baseball Gear. Fanatics.com is your ultimate College Apparel store, bringing Students, Fans and Alumni all the College Gear they could ever need. With a huge selection of College Jerseys, College Clothing, NCAA Sweatshirts and T-Shirts, every die-hard fan or alumnus can find a way to show their school pride. We offer College Sportswear for over 500 schools, so you can be sure you're checking out the largest collection of officially licensed College Gear and NCAA Merchandise anywhere. Our NCAA Shop houses an endless variety of College Jerseys so you'll be totally set for every season. 

                  Shop